In [46]:
############## import packages ##############
import pandas as pd
import numpy as np
import gurobipy as gp
from tqdm import tqdm
from joblib import Parallel, delayed
import warnings
warnings.filterwarnings('ignore')
pd.reset_option('display.float_format')

In [47]:
############## import data ##############
df=pd.read_excel('AirlineCallVolumes.xlsx',sheet_name='Data')
#make hour column as row index instead of a column
df.drop(columns=['Hour'],inplace=True)
df.index.name='Hour' 
df.head()

,Sunday,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday
Hour,,,,,,,
0,7.3,7.3,8.5,7.2,8.2,14.3,7.8
1,9.8,18.3,11.5,10.8,10.0,12.3,9.6
2,14.8,26.5,17.5,27.4,21.4,28.0,12.0
3,20.5,46.5,31.5,40.8,32.6,45.5,24.0
4,41.0,68.8,58.0,78.8,65.8,78.0,47.0


In [48]:
############## define functions ##############
def service_capacity(systemsize,agents,arrival,service):

    service_time=1/service

    totalservice = np.zeros(systemsize+1)
    i = 1
    while i<=systemsize:
        totalservice[i] = min(i, agents) * service
        i += 1
    
    state_coeff = np.zeros(systemsize+1)
    state_prob = np.zeros(systemsize+1)
    state_coeff[0]=1.0
    state_coeff[1] = arrival / service
    i=2
    while i <= systemsize:
        state_coeff[i] = (arrival + totalservice[i-1]) * state_coeff[i-1] 
        state_coeff[i] -= (arrival * state_coeff[i-2])
        state_coeff[i] *= (1/totalservice[i]) 
        i += 1

    coeffsum = np.sum(state_coeff)

    for i in range(systemsize+1):
        state_prob[i] = state_coeff[i]/coeffsum
        
    Line = 0.
    for i in range(systemsize): Line += (i+1)*state_prob[i+1]
    actual_arrival = arrival*(1-state_prob[systemsize])
    wait =   Line / actual_arrival
    line_wait = wait - service_time
    hold_prob = 0.
    i = agents
    while i<= systemsize: 
        hold_prob += state_prob[i]
        i += 1

    busy_prob = state_prob[systemsize]


    df=pd.DataFrame(data=[agents,Line,3600*actual_arrival,wait,line_wait,hold_prob,busy_prob])
    df=df.T
    df.columns=['agent_num','total_cus','actual_arrival','total_time','line_wait','hold_prob',
                'busy_prob']

    return df

In [49]:
def agent_min(systemsize,arrival,service,waiting_constraint):

    table=Parallel(n_jobs=-1,backend='loky') \
                        (delayed(service_capacity)(systemsize=500,agents=agents,arrival=arrival,service=service)
                        for agents in range(1,systemsize+1))
    df=pd.concat(table)

    return df[df['line_wait']<=waiting_constraint]['agent_num'].min()

### Question B

In [50]:
calls_per_hour = 168
arrival = calls_per_hour/3600   # arrivals per second
service_time = 372
service = 1 / service_time  # number of call served per second

In [51]:
agent_min(systemsize=500,arrival=arrival,service=service,waiting_constraint=45)

21.0

In [52]:
service_capacity(systemsize=500,agents=21,arrival=arrival,service=service)

,agent_num,total_cus,actual_arrival,total_time,line_wait,hold_prob,busy_prob
0,21.0,18.837199,168.0,403.654258,31.654258,0.309735,-1.101027e-16


In [53]:
service_capacity(systemsize=100,agents=20,arrival=arrival,service=service)
#confirm the minimum number of server is 21. 20 would not work

,agent_num,total_cus,actual_arrival,total_time,line_wait,hold_prob,busy_prob
0,20.0,20.255173,167.999882,434.039737,62.039737,0.440339,7.013635e-07


In [54]:
agent_df=df.applymap(lambda arrival: agent_min(systemsize=100,arrival=arrival/3600,service=service,
                    waiting_constraint=45))
agent_df

,Sunday,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday
Hour,,,,,,,
0,3.0,3.0,3.0,3.0,3.0,4.0,3.0
1,3.0,4.0,3.0,3.0,3.0,3.0,3.0
2,4.0,5.0,4.0,5.0,5.0,5.0,3.0
3,4.0,8.0,6.0,7.0,6.0,7.0,5.0
4,7.0,10.0,9.0,11.0,10.0,11.0,8.0
5,8.0,15.0,13.0,14.0,14.0,15.0,10.0
6,10.0,20.0,16.0,16.0,17.0,20.0,12.0
7,12.0,19.0,17.0,18.0,19.0,20.0,13.0
8,17.0,19.0,17.0,17.0,20.0,21.0,14.0


### Question C

In [55]:
work_schedule=np.array([1,1,1,1,0,1,1,1,1])

In [56]:
daily_shifts={}
for i in range(16):
    daily_shifts[i]=np.concatenate([np.zeros(i),work_schedule,np.zeros(24-i-9)])
for i in range(16,24):
    daily_shifts[i]=np.concatenate([work_schedule[24-i:],np.zeros(24-9),work_schedule[:24-i]])
daily_shifts

{0: array([1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.]),
 1: array([0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.]),
 2: array([0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.]),
 3: array([0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.]),
 4: array([0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.]),
 5: array([0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.]),
 6: array([0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.]),
 7: array([0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0.,
        0., 0., 0., 0., 0., 0., 0.]),
 8: array([0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1

In [57]:
a=np.array(list(daily_shifts.values()))

In [58]:
model=gp.Model()

#Xi: number of servers in shift i
x=model.addVars(24,lb=0,name="X",vtype=gp.GRB.INTEGER)

model.update()

In [59]:
model.setObjective(x.sum(),gp.GRB.MINIMIZE)

In [60]:
for j in range(24):
    num_agents=0
    for i in range(24):
        num_agents+=x[i]*a[i][j]
    model.addConstr(num_agents>=int(agent_df['Sunday'][j])) 

In [61]:
model.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 24 rows, 24 columns and 192 nonzeros
Model fingerprint: 0x3ef09917
Variable types: 0 continuous, 24 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 2e+01]
Found heuristic solution: objective 50.0000000
Presolve time: 0.00s
Presolved: 24 rows, 24 columns, 192 nonzeros
Variable types: 0 continuous, 24 integer (0 binary)

Root relaxation: objective 3.350000e+01, 32 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   33.50000    0    7   50.00000   33.50000  33.0%     -    0s
H    0     0                      34.0000000   33.50000  1.47%     -    0s
     0     0 

In [62]:
model.ObjVal

34.0

In [63]:
model.printAttr(['X'])


    Variable            X 
-------------------------
        X[0]            3 
        X[1]            2 
        X[3]            4 
        X[4]            1 
        X[5]            1 
        X[7]            5 
        X[8]            2 
        X[9]            2 
       X[11]            6 
       X[13]            1 
       X[14]            2 
       X[15]            5 


In [64]:
shift_numb=0
for i in range(24):
    if model.getVars()[i].X>0:
        shift_numb+=1
print(shift_numb)

12


# Question D

In [65]:
work_schedule=np.array([1,1,1,1,0,1,1,1,1])

In [66]:
weekly_shifts={}
for i in range(7*24-8):
    weekly_shifts[i]=np.concatenate([np.zeros(i),work_schedule,np.zeros(7*24-9-i)])

weekly_shifts

{0: array([1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 1: array([0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 

In [67]:
a=np.array(list(weekly_shifts.values()))

In [68]:
model=gp.Model()
x=model.addVars(7*24-8,lb=0,name="X",vtype=gp.GRB.INTEGER)
model.update()

In [69]:
model.setObjective(x.sum(),gp.GRB.MINIMIZE)

In [70]:
for j in range(7*24):
    num_servers=0
    for i in range(7*24-8):
        num_servers+=x[i]*a[i][j]
    model.addConstr(num_servers>=server_df.to_numpy().T.flatten()[j])

In [71]:
model.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 168 rows, 160 columns and 1280 nonzeros
Model fingerprint: 0xd3ad3560
Variable types: 0 continuous, 160 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 2e+01]
Found heuristic solution: objective 303.0000000
Presolve removed 4 rows and 0 columns
Presolve time: 0.00s
Presolved: 164 rows, 160 columns, 1272 nonzeros
Variable types: 0 continuous, 160 integer (0 binary)

Root relaxation: objective 2.780000e+02, 218 iterations, 0.02 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  278.00000    0   56  303.00000  278.00000  8.25%     -    0s
H    0     0                     285.000

In [72]:
model.ObjVal

280.0

In [73]:
model.printAttr(['X'])


    Variable            X 
-------------------------
        X[0]            3 
        X[1]            3 
        X[2]            2 
        X[3]            3 
        X[4]            1 
        X[5]            3 
        X[6]            1 
        X[8]            3 
        X[9]            4 
       X[10]            1 
       X[11]            5 
       X[12]            2 
       X[13]            2 
       X[15]            4 
       X[19]            1 
       X[24]            4 
       X[26]            1 
       X[27]            5 
       X[28]            5 
       X[29]            3 
       X[30]            4 
       X[31]            3 
       X[33]            2 
       X[35]            4 
       X[36]            2 
       X[37]            4 
       X[39]            3 
       X[46]            2 
       X[48]            1 
       X[49]            4 
       X[51]            3 
       X[52]            4 
       X[53]            4 
       X[55]            7 
       X[59]            7 
 

# Question E

In [74]:
shift_num=0
for i in range(7*24-8):
    if model.getVars()[i].X>0:
        shift_num+=1
print(shift_num)

98


# Question G

In [36]:
agent_df=df.applymap(lambda arrival: agent_min(systemsize=100,arrival=arrival/3600,service=service,
                    waiting_constraint=30))
agent_df

,Sunday,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday
Hour,,,,,,,
0,3.0,3.0,3.0,3.0,3.0,4.0,3.0
1,3.0,4.0,3.0,3.0,3.0,4.0,3.0
2,4.0,5.0,4.0,6.0,5.0,6.0,4.0
3,5.0,8.0,6.0,7.0,6.0,8.0,5.0
4,7.0,11.0,9.0,12.0,10.0,12.0,8.0
5,8.0,16.0,13.0,14.0,15.0,15.0,10.0
6,10.0,20.0,17.0,17.0,17.0,21.0,13.0
7,12.0,19.0,18.0,18.0,20.0,21.0,14.0
8,17.0,19.0,18.0,18.0,21.0,22.0,14.0


In [37]:
model=gp.Model()

# Number of agents in shift i, Xi
x=model.addVars(7*24-8,lb=0,name="X",vtype=gp.GRB.INTEGER)

model.update()

model.setObjective(x.sum(),gp.GRB.MINIMIZE)

for j in range(7*24):
    num_agents=0
    for i in range(7*24-8):
        num_agents+=x[i]*a[i][j]
    model.addConstr(num_agents>=agent_df.to_numpy().T.flatten()[j])

model.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 168 rows, 160 columns and 1280 nonzeros
Model fingerprint: 0x35c806ad
Variable types: 0 continuous, 160 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 2e+01]
Found heuristic solution: objective 283.0000000
Presolve removed 4 rows and 0 columns
Presolve time: 0.01s
Presolved: 164 rows, 160 columns, 1273 nonzeros
Variable types: 0 continuous, 160 integer (0 binary)

Root relaxation: objective 2.607500e+02, 187 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  260.75000    0   45  283.00000  260.75000  7.86%     -    0s
H    0     0                     264.000

In [38]:
model.ObjVal

262.0

In [39]:
shift_num=0
for i in range(7*24-8):
    if model.getVars()[i].X>0:
        shift_num+=1
print(shift_num)

94


# Question H

In [40]:
agent_df=df.applymap(lambda arrival: agent_min(systemsize=500,arrival=arrival/3600,service=service,
                    waiting_constraint=15))
agent_df

,Sunday,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday
Hour,,,,,,,
0,3.0,3.0,3.0,3.0,3.0,4.0,3.0
1,4.0,5.0,4.0,4.0,4.0,4.0,4.0
2,4.0,6.0,5.0,6.0,5.0,6.0,4.0
3,5.0,9.0,7.0,8.0,7.0,8.0,6.0
4,8.0,11.0,10.0,12.0,11.0,12.0,9.0
5,9.0,17.0,14.0,15.0,16.0,16.0,11.0
6,11.0,21.0,18.0,17.0,18.0,22.0,14.0
7,13.0,20.0,19.0,19.0,21.0,22.0,15.0
8,18.0,20.0,19.0,19.0,22.0,23.0,15.0


In [41]:
model=gp.Model()

x=model.addVars(7*24-8,lb=0,name="X",vtype=gp.GRB.INTEGER)

model.update()

model.setObjective(x.sum(),gp.GRB.MINIMIZE)

for j in range(7*24):
    num_agents=0
    for i in range(7*24-8):
        num_agents+=x[i]*a[i][j]
    model.addConstr(num_agents>=agent_df.to_numpy().T.flatten()[j])

model.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 168 rows, 160 columns and 1280 nonzeros
Model fingerprint: 0xd3ad3560
Variable types: 0 continuous, 160 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 2e+01]
Found heuristic solution: objective 303.0000000
Presolve removed 4 rows and 0 columns
Presolve time: 0.00s
Presolved: 164 rows, 160 columns, 1272 nonzeros
Variable types: 0 continuous, 160 integer (0 binary)

Root relaxation: objective 2.780000e+02, 218 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  278.00000    0   56  303.00000  278.00000  8.25%     -    0s
H    0     0                     285.000

In [42]:
model.ObjVal

280.0

In [43]:
shift_num=0
for i in range(7*24-8):
    if model.getVars()[i].X>0:
        shift_num+=1
print(shift_num)

98
